# 夏目漱石のテキストでGPT訓練

このノートブックでは、夏目漱石のテキストを使用してGPTモデルを訓練します。
nanoGPTと同様のアプローチで、文字レベルの言語モデルを作成します。

## 必要なライブラリのインポート

In [ ]:
import os
import re
import torch
import requests
from bs4 import BeautifulSoup
from llm_from_scratch.gpt import (
    GPT, GPTConfig, SimpleTokenizer,
    create_dataloaders, GPTTrainer
)

## 夏目漱石のテキストダウンロード関数

青空文庫から夏目漱石の「吾輩は猫である」をダウンロードし、テキストを前処理します。

In [ ]:
def download_soseki():
    """夏目漱石のテキストをダウンロード。"""
    url = "https://www.aozora.gr.jp/cards/000148/files/789_14547.html"

    # データディレクトリを作成
    os.makedirs("data/soseki", exist_ok=True)

    # ダウンロードして処理
    filepath = "data/soseki/input.txt"
    if not os.path.exists(filepath):
        print("夏目漱石のテキストをダウンロード中...")
        response = requests.get(url)
        response.encoding = 'shift_jis'  # 青空文庫はShift_JIS

        # BeautifulSoupでHTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')

        # 本文を抽出（青空文庫の構造に基づく）
        main_text = soup.find('div', class_='main_text')
        if main_text:
            text = main_text.get_text()
        else:
            # フォールバック：bodyからテキストを抽出
            text = soup.get_text()

        # 青空文庫の注記や記号を除去
        text = re.sub(r'［＃.*?］', '', text)  # 注記を除去
        text = re.sub(r'《.*?》', '', text)    # ルビを除去
        text = re.sub(r'｜', '', text)        # 縦線を除去
        text = re.sub(r'　', ' ', text)       # 全角スペースを半角に
        text = re.sub(r'\n+', '\n', text)     # 連続する改行を一つに
        text = text.strip()

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"ダウンロード完了: {filepath}")

    # テキストを読み込み
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    print(f"データセットサイズ: {len(text)} 文字")
    return text

## データの取得

夏目漱石のテキストをダウンロードして確認します。

In [ ]:
# データセットをダウンロードして読み込み
text = download_soseki()

# テキストの一部を表示
print("テキストの最初の500文字:")
print(text[:500])

## トークナイザーの作成

文字レベルのトークナイザーを作成し、語彙サイズを確認します。

In [ ]:
# 文字レベルのトークナイザーを作成
tokenizer = SimpleTokenizer(text)
print(f"語彙数: {tokenizer.vocab_size} ユニーク文字")

# 語彙の一部を表示
print("\n語彙の一部:")
chars = list(tokenizer.char_to_idx.keys())[:20]
print(chars)

## データローダーの作成

訓練用と検証用のデータローダーを作成します。

In [ ]:
# データローダーを作成
train_loader, val_loader = create_dataloaders(
    text, tokenizer,
    block_size=256,      # コンテキスト長
    batch_size=64,       # バッチサイズ
    train_split=0.9
)

print(f"訓練データバッチ数: {len(train_loader)}")
print(f"検証データバッチ数: {len(val_loader)}")

## GPT-2標準設定の参考

**GPT-2の各モデルサイズの仕様:**

| モデル | パラメータ | n_embd | n_layer | n_head | vocab_size |
|--------|------------|--------|---------|--------|------------|
| Small  | 124M       | 768    | 12      | 12     | 50257      |
| Medium | 355M       | 1024   | 24      | 16     | 50257      |
| Large  | 774M       | 1280   | 36      | 20     | 50257      |
| XL     | 1.5B       | 1600   | 48      | 25     | 50257      |

**推奨設定:**
- **リソース限定**: n_embd=512, n_layer=8, n_head=8 (~40M params)
- **標準学習**: GPT-2 Small設定 (124M params)
- **本格学習**: GPT-2 Medium以上

下記では軽量な設定を使用します。

## モデル設定とモデル作成

nanoGPTのshakespeareと同様の設定でGPTモデルを作成します。

In [ ]:
# モデル設定（nanoGPTのshakespeareと同様）
config = GPTConfig(
    vocab_size=tokenizer.vocab_size,
    n_embd=384,          # 埋め込み次元
    n_layer=6,           # レイヤー数
    n_head=6,            # アテンションヘッド数
    block_size=256,      # コンテキストウィンドウ
    dropout=0.2
)

print(f"モデルサイズ: 約{config.get_model_size():.2f}Mパラメータ")

# モデルを作成
model = GPT(
    vocab_size=config.vocab_size,
    n_embd=config.n_embd,
    n_layer=config.n_layer,
    n_head=config.n_head,
    block_size=config.block_size,
    dropout=config.dropout
)

print("モデル作成完了")

## トレーナーの設定

モデルの訓練に必要なトレーナーを設定します。

In [ ]:
# トレーナーを作成
trainer = GPTTrainer(
    model, train_loader, val_loader,
    learning_rate=1e-3,
    weight_decay=0.1,
    warmup_steps=100,
    max_steps=5000,      # 5000ステップ訓練
    grad_clip=1.0
)

print("トレーナー設定完了")
print(f"デバイス: {trainer.device}")

## モデルの訓練

GPTモデルを夏目漱石のテキストで訓練します。

In [ ]:
# モデルを訓練
print("訓練開始...")
losses = trainer.train(log_interval=100, eval_interval=500)
print("訓練完了!")

## テキスト生成のテスト

訓練されたモデルを使用して、夏目漱石風のテキストを生成します。

In [ ]:
print("夏目漱石風のテキストを生成中...")
print("=" * 50)

model.eval()

# プロンプトで開始
prompts = [
    "吾輩は猫である",
    "明治",
    "東京",
    "先生"
]

for prompt in prompts:
    print(f"\nプロンプト: {prompt}")
    print("-" * 30)
    
    # プロンプトをエンコード
    prompt_tokens = torch.tensor(
        tokenizer.encode(prompt),
        dtype=torch.long
    ).unsqueeze(0).to(trainer.device)
    
    # テキストを生成
    with torch.no_grad():
        generated = model.generate(
            prompt_tokens,
            max_new_tokens=200,
            temperature=0.8,
            top_k=40
        )
    
    generated_text = tokenizer.decode(generated[0].cpu().numpy())
    print(generated_text)

## モデルの保存

訓練されたモデルを保存します。

In [ ]:
# モデルを保存
os.makedirs("models", exist_ok=True)
checkpoint_path = "models/soseki_gpt_checkpoint.pt"
trainer.save_checkpoint(checkpoint_path)
print(f"モデル保存: {checkpoint_path}")

## 訓練損失の可視化（オプション）

訓練過程の損失をプロットして確認します。

In [ ]:
import matplotlib.pyplot as plt

# 損失をプロット
if losses:
    train_losses = [loss['train_loss'] for loss in losses if 'train_loss' in loss]
    val_losses = [loss['val_loss'] for loss in losses if 'val_loss' in loss]
    
    plt.figure(figsize=(12, 4))
    
    # 訓練損失
    plt.subplot(1, 2, 1)
    plt.plot(train_losses)
    plt.title('Training Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # 検証損失
    if val_losses:
        plt.subplot(1, 2, 2)
        plt.plot(val_losses)
        plt.title('Validation Loss')
        plt.xlabel('Evaluation Step')
        plt.ylabel('Loss')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("損失データが見つかりません")

## まとめ

このノートブックでは以下を実行しました：

1. 夏目漱石のテキストを青空文庫からダウンロード
2. テキストの前処理（注記・ルビの除去など）
3. 文字レベルトークナイザーの作成
4. データローダーの作成
5. GPTモデルの設定と作成
6. モデルの訓練
7. テキスト生成のテスト
8. モデルの保存

訓練されたモデルは夏目漱石のスタイルを学習し、類似したテキストを生成できるようになります。